# GDSC Training Session - Advanced Modelling

***
<a id='Introduction'></a>
# Introduction

After creating our first simple model in the last training session we this time want to get into a more sophisticated technique to approach our task. This includes not only a more complex kind of model but also the steps to include bigger parts of our data into the training. This scale up will come in two ways:
* include data from a longer time frame into the training
* prepare data from different sensors as input for the model

So, considering the general framework [CRISP-DM](https://www.sv-europe.com/crisp-dm-methodology/) we will this time focus on the Data Preparation and Modelling steps. Once again, we will not end up with the perfect model but provide further ideas how to improve on these aspects.

Now, lets get started preparing the data and building our next model!!

*Notes:*
- *If this is your first Jupyter Notebook you might want to read [a general introduction](https://realpython.com/jupyter-notebook-introduction/) first.*
- *Do not be afraid to ask (and answer) questions! Check out the [teams channel](https://teams.microsoft.com/l/team/19%3a4017a2e9af4942e7aa157d6ec9d751b4%40thread.skype/conversations?groupId=7d77d672-dff1-4c9f-ac55-3c837c1bebf9&tenantId=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61).*
- *Material and recordings from last weeks session in this [teams post](https://teams.microsoft.com/l/message/19:4017a2e9af4942e7aa157d6ec9d751b4@thread.skype/1614516939664?tenantId=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61&groupId=7d77d672-dff1-4c9f-ac55-3c837c1bebf9&parentMessageId=1614516939664&teamName=Global%20Data%20Science%20Challenge%20-%20Public&channelName=General&createdTime=1614516939664).*

## Table of contents:
[1. Introduction](#Introduction) <br>
[2. Recap - Task of the Challenge](#Recap) <br>
[3. Data Preparation](#DataPrep) <br>
&emsp; [3.1. Harmonizing the Sample Frequency](#Harmonize) <br>
&emsp; [3.2. Data Stacking](#Stacking) <br>
[4. Advanced Modelling](#Modelling) <br>
&emsp; [4.1. Introduction](#ModelIntro) <br>
&emsp; [4.2. Autoencoder Architecture](#Architecture) <br>
&emsp; [4.3. Feeding big amounts of data to the model](#DataGenerator) <br>
&emsp; [4.4. Leveraging Deep Leraning Frameworks on SageMaker](#Frameworks) <br>
&emsp; [4.5. Creating a Training Job](#TrainingJob) <br>
&emsp; [4.6. Inference](#Inference) <br>
[5. Wrap Up](#Epilogue) <br>


***
<a id='Recap'></a>
# Recap - What we learned so far
In last weeks session we performed exploratory data analysis (EDA) on the EK60 echosounder and point sensor data. We asked and answered questions about our data regarding the overall shape as well as created visualizations in order to manually gather insights from these two data sources.

Also, we created a basic model based on a built-in AWS SageMaker algorithm - Random Cut Forrest (RCF) - to score our data points on their likelihood to be an anomaly. The decision if a scored datapoint is deemed an anomaly was based on a threshold we set after gathering some statistics on the distribution of the score values.

Finally, we submitted the found anomalies as our first contribution to the Global Data Science Challenge.

***
<a id='Recap'></a>
# Data Preparation

Since our task is to find anomalies across the various sensor readings we might not just look at the data of each sensor individually but find a way to combine it. This way we can provide it to a model which has the chance to pick up on correlations between the sensors. For example we would expect to see higher speed values near the ocean surface in the ADCP when there is more scatter on the ocean surface visible in the echogram due to increased wave activity. This could for example be due to the weather. This might also be picked up by the hydrophone. 

<a id='Harmonize'></a>
## Harmonizing the Sample Frequency

In order to combine the data one important step is to harmonize the sample frequency of the different sensors to a common sample rate. As you probably recall from our session last week and also validated in your own analysis this part has already been taken care of for the tabular data provided in S3. There we already have one data point per minute whereever data is available. Here again the overview of performed steps to get there:
* ADCP
    * Extraction of current speed and direction from the .nc files using [NetCDF4](https://pypi.org/project/netCDF4/) and [xarray](http://xarray.pydata.org/en/stable/)
    * Linear interpolation of the data to provide one data point per minute sample rate - before roughly one every ten minutes
    * Store data in CSV tables with one file per day of data and one data point per minute - speed and direction in one table, check the column names
    
* EK60 - Echosounder
    * Excration of the volume backscatter strengh info from the .raw files using [PyEchoLab](https://github.com/CI-CMG/pyEcholab)
    * Aggregation of the data using a mean to one data point per minute - before roughly one data point every 4 seconds
    * Cutoff of values further away from the sensor than 300 meters, so data above the ocean surface
    * Store data in CSV tables with one file per day of data and one data point per minute
* Hydrophone
    * Downsampling the .wav audio files to 12kHz maximum frequency using [librosa](https://librosa.org/doc/latest/index.html) - our research partners assured us that the information loss from this is negligable (Note: this is the state we provided to you in the *raw/* folder since the pure .wav files have an even bigger volume)
    * Performing [Long Term Spectral Average](https://libraries.io/github/tryan/LTSA) on the data to have it available in the frequeny vs. time domain
    * Store data in CSV tables with one file per day of data and one data point per minute
* Point Sensors - Temperature, Salinity, Depth, ....
    * Raw data already provided in CSV format with roughly hourly measurements
    * Linear interpolation of the data to provide one data point per minute sample rate - before roughly one every hour
    * Store data in CSV tables with one file per day of data and one data point per minute

Since this first part of the data preparation was already performed within this tutorial we can come right away the the stacking of the data. This we will do in the following.

<a id='Stacking'></a>
## Data Stacking
We will now have a look at how to stack our data from the different sensors together to get one combined measurement point that contains the information of all our sensors. We will only show how it works for data of one day. Scaling this out for every day and possibly doing additional preprocessing on the data will be up to you.

We start by downloading the CSV files from each sensor for a particular day. We can check the data availability for the day quickly via the AWS CLI:

In [ ]:
!aws s3 ls s3://gdsc4-eu/data/adcp/tabular/2015-05-25

In [ ]:
import boto3
from botocore.exceptions import ClientError
from pathlib import Path

def get_file_list_from_s3(bucket_name, prefix):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    file_list = list(bucket.objects.filter(Prefix=prefix))
    return [element.key for element in file_list]

def download_file(bucket_name, file_path, local_output_path):

    s3 = boto3.resource('s3')
    file_name = file_path.split('/')[-1]
    try:
        s3.Bucket(bucket_name).download_file(
            str(file_path), str(Path(local_output_path, file_name))
        )
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

We will just download the data we want to use for the stacking.

In [ ]:
bucket = "gdsc4-eu"
file_date = "2015-05-25"

download_file(bucket, f"data/adcp/tabular/{file_date}.csv", Path("data/adcp"))
download_file(bucket, f"data/ek60/tabular/{file_date}.csv", Path("data/ek60"))
download_file(bucket, f"data/hydrophone/tabular/{file_date}.csv", Path("data/hydrophone"))
download_file(bucket, f"data/point_sensors/tabular/{file_date}.csv", Path("data/point_sensors"))

Next we load all the CSV files into dataframe making sure we have a DatetimeIndex in place for all of them.

In [ ]:
import pandas as pd

file_date = "2015-05-25"
adcp_data = pd.read_csv(Path(f"data/adcp/{file_date}.csv"), index_col=0, sep=',', parse_dates=True, header=0)
ek60_data = pd.read_csv(Path(f"data/ek60/{file_date}.csv"), index_col=0, sep=',', parse_dates=True, header=0)
hydrophone_data = pd.read_csv(Path(f"data/hydrophone/{file_date}.csv"), index_col=0, sep=',', parse_dates=True, header=0)
point_sensors_data = pd.read_csv(Path(f"data/point_sensors/{file_date}.csv"), index_col=0, sep=',', parse_dates=True, header=0)

Let's briefly check the format of each of the data sources again:

In [ ]:
adcp_data.head()

In [ ]:
ek60_data.head()

In [ ]:
hydrophone_data.head()

In [ ]:
point_sensors_data.head()

On a first glance most of the data looks good. The point sensors however seem to only contain info for turbidity. For now we will continue down our path to combine the data but this is an issue that needs to be adressed before the data is fed to a model for training, because most algorithms cannot handle this issue on their own. 

So let's join our data, and [join](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join) is the keyword here. Since pandas dataframes are basically tables very much like tables in databases we can make use of the methods that are very common in relational database interactions. All our tables have a common index, the timestamps. We can use this index to perform the join operation on. Since we have data available for each minute of the day in each table the kind of join does not matter in our particular case.

In [ ]:
all_sensor_data = adcp_data.join(ek60_data, how="outer").join(hydrophone_data, how="outer").join(point_sensors_data, how="outer")
all_sensor_data

> Exercise: What can you do to validate that the resulting data frame contains all the information (and also not more as) you want it to have? 

> Exercise: The approach described above assumes that the data from each sensor is available for a given day. As we have seen in our EDA, this is not the case for several periods within our time frame of interest. How can this be mitigated?

> Exercise: try out different join methods (inner, outer, left, right, ...) when you have tables that do not contain 1440 data points per day and you therefore need to make up for the missing data in one of the sensor tables. What are the implications of the join method?

This is already the end of our data preparation steps. We did not mitigate the obvious issues that arise from our data gaps or incomplete data on a particular day. Nonetheless this provide you some idea on how to tackle these problems. And do not forget, share your ideas and discuss with the community how to best approach this further!

***
<a id='Modelling'></a>
# Advanced Modelling

<a id='ModelIntro'></a>
## Introduction

An [autocoder](https://en.wikipedia.org/wiki/Autoencoder) is a type of artificial neural network used to learn efficient data codings in an unsupervised manner. The aim of an autoencoder is to learn a representation (encoding) for a set of data, typically for dimensionality reduction, so in a compressed form, by training the network to ignore signal “noise”. Along with the reduction side, a reconstructing side is learned, where the autoencoder tries to generate from the reduced encoding a representation as close as possible to its original input, hence its name.

<img src="./notebook_images/Autoencoder_schema.png" alt="autoencoder" width="400"/>

The idea of autoencoders has been popular in the field of neural networks for decades. The first applications date to the 1980s. Their most traditional application was dimensionality reduction or feature learning, but the autoencoder concept became more widely used for learning generative models of data. Some of the most powerful AIs in the 2010s involved sparse autoencoders stacked inside deep neural networks.

**So how does this relate to anomaly detection like we are aiming to do?**

The mentioned application - dimensionality reduction - holds the key to this question. Once we are able to identify the main patterns the outliers can be revealed. For dimensionality reduction we could of course use tools like [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) but many traditional techniques like PCA suffer from the fact that they rely on linear transformations. In contrast, the autoencoder techniques can perform non-linear transformations and thus show their merits when the data problems are complex and non-linear.

The autoencoder approach for our problem follows similar lines as we used for the RCF, just the way the anomaly score is determined changes. So what we want to do is the following:
1. Train the autoencoder model to be able to reconstruct our data
2. Provide our data to the model during inference and receive the reconstructed version
3. Calculate the reconstruction error, which will serve as our "anomaly score"
4. Analyze the reconstruction errors to define a threshold value
5. Identify all data points that have a reconstruction error above the set threshold value

This process hinges on the assumption that the trained autoencoder model will be able to very well reconstruct the data for normal situations, because it has seen lots of examples of those. The reconstructions of anomalies on the other hand will be worse and hence result in a higher reconstruction error.

So let's start by looking at the overall network architecture before diving into some details:

<a id='Architecture'></a>
## Model Architecture
The model architecture introduced here is quite simple. It can of course be further enhanced and tuned but for the tutorial here we want to keep it simple and only highlight the essentials for this approach. So here the implementation of the model architecture:

```python
def model(x_train, x_test, epochs, window_size, feature_dim, batch_size=1):
    """Generate a simple model"""
    
    input_data = keras.Input(shape=(window_size, feature_dim, 1), batch_size=batch_size)

    x = layers.Conv2D(16, (4, 4), activation='relu', padding='same')(input_data)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (4, 4), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (4, 4), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = layers.Conv2D(8, (4, 4), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(8, (4, 4), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(16, (4, 4), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (4, 4), activation='sigmoid', padding='same')(x)

    autoencoder = keras.Model(input_data, decoded)
    autoencoder.summary()
    autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')

    autoencoder.fit(x_train, epochs=epochs, validation_data=x_test)

    return autoencoder
```

This implementation not only contains the definition of the model architecture but also sets the parameters necessary to compile the model and start the fitting of the model. For example we set our optimizer (Adam), the loss function (mean squarred error (mse)) to optimize and the for deep learning crucial hyperparameter for the learning rate.

The input for the model will be our data points, hence the *feature_dim* parameter. Also, we want provide some context for a particular data point to the model. So we introduce a *window_size* which combines multiple data points that follow each other on the time dimension to one data point. This way we have a start to introduce the time dimension of our problem to the model. Finally, in deep learning you want to provide your model not with all the data at once and also not one data point at a time. Hence, the data is fed to the model in batches, smaller sample collections of your data (see this [blog](https://medium.com/analytics-vidhya/when-and-why-are-batches-used-in-machine-learning-acda4eb00763) for a few more details on the whens and whys of using batches). 

With a *window_size* of 64, a *batch_size* of 128 and *feature_dim* of 1600 this will yield the following structure:

This is still a very small model but it highlights the basic principles of the autoencoder. Three layers of [convolutions](https://keras.io/api/layers/convolution_layers/convolution2d/) and [max-pooling](https://keras.io/api/layers/pooling_layers/max_pooling2d/) each for the encoder to get to the compressed data representation (i.e. with reduced dimensionality) followed by three layers of again convolution and [up-sampling](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) each for the decoder (i.e. to reconstruct the data from the compressed representation) that provides us the same output dimensions as we provided as input.

We will not cover all the details of the model architecture as part of this tutorial. For this we can recommend the very good blog article on [Convolutional Autoencoders for Image Noise Reduction](https://towardsdatascience.com/convolutional-autoencoders-for-image-noise-reduction-32fce9fc1763). In particular the section on "How Does the Convolutional Autoencoders Work?". For the technical aspects and implementation details we recommend to check the [Keras API reference](https://keras.io/api/) and also the [Tensorflow tutorial on autoencoders](https://www.tensorflow.org/tutorials/generative/autoencoder).
  
The model architecture however is only part of the picture. We also need to make sure our model is provided with the necessary input data and for a neural network we love to have big amounts of data to train on. On the next section we will see how this can be achieved.

<a id='DataGenerator'></a>
## Feeding big amounts of data to the model
In many tutorials and example implementations introducing you to machine learning and data science (like this generally very nice one on [timeseries anomaly detection using an Autoencoder](https://keras.io/examples/timeseries/timeseries_anomaly_detection/) or ) you will see that the data used to train the model is simply loaded **as a whole** into a Pandas dataframe or Numpy array and then provided to the model for training. This however assumes that the entire amount of data that is used can fit into memory on the machine that is used for training. Since the amount of data we are using is way too big to do so we will have to use a much smarter way to provide the training data to our model. The important point to realize is that the training process is never using all the data at once. It only needs parts of it at a time. Due to this fact TensorFlow/Keras provides the option of implementing a so called DataGenerator (see this nice [blog article](https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly) as an introduction) based on the [Keras Sequence](https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence) implementation. The DataGenerator can be passed to the fit method of the estimator and will squentially load only the part of the data that is currently needed for training and provide it during training.

For our problem this DataGenerator will therefore load a certain number of rows of and across the daily CSV files. Here is the implementation to do so:

```python
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
import math
from pathlib import Path
import itertools
import boto3

class DataGeneratorFromCSV(Sequence):
    def __init__(self, file_names, window_size, feature_dim, batch_size, default_s3_location="data/ek60/tabular"):
        self.file_names = file_names
        self.s3_location = default_s3_location
        self.window_size = window_size
        self.feature_dim = feature_dim
        self.batch_size = batch_size
        self.local_meta_info_path = Path("meta_info")
        self.local_meta_info_path.mkdir(parents=True, exist_ok=True)
        
        self.record_book = self.setup_record_book(file_names, default_s3_location)
        
        self.total_number_of_records = len(self.record_book)
    
    @staticmethod
    def setup_record_book(file_names, s3_location):
        
        for file_name in file_names:
            DataGeneratorFromCSV.download_file(
                "gdsc4-eu", 
                f"{s3_location}/meta_info_{file_name.stem}.txt", 
                Path.cwd()
            )
        file_infos = {
            file_name: DataGeneratorFromCSV.extract_number_of_records(
                Path(Path.cwd(), f"meta_info_{file_name.stem}.txt")
            ) for file_name in file_names
        }
        total_number_of_records = sum(file_infos.values())

        record_table = {
            'overall_record_id': [entry for entry in range(0, total_number_of_records)],
            'file_record_id': list(itertools.chain.from_iterable(
                [range(0, file_infos[file_name]) for file_name in file_infos.keys()]
            )),
            'file_name': list(itertools.chain.from_iterable(
                [itertools.repeat(file_name, file_infos[file_name]) for file_name in file_infos.keys()]
            )),
        }
        record_book = pd.DataFrame.from_dict(record_table)         
        return record_book.set_index('overall_record_id')
        
        
    def __len__(self):
        return math.floor(
            self.total_number_of_records / (self.batch_size * self.window_size)
        ) - 1
    
    def __getitem__(self, index):
        from_record = index * self.batch_size * self.window_size
        to_record = (index + 1) * self.batch_size * self.window_size
        csv_files = self.record_book[from_record:to_record]["file_name"].unique()
        from_record_info = self.record_book.iloc[from_record]
        to_record_info = self.record_book.iloc[to_record]
        
        if len(csv_files) == 1:
            df = pd.read_csv(from_record_info.file_name)
            df = df[from_record_info.file_record_id:to_record_info.file_record_id]
            batch = self.perform_preprocessing(df)
            batch = batch.reshape([self.batch_size, self.window_size, self.feature_dim, 1])
        else:
            data_collection = []
            for csv_file in csv_files:
                data = pd.read_csv(csv_file)
                data.columns = [f"{el}" for el in range(0, data.shape[1])] # enforce exactly matching column names
                if csv_file == csv_files[0]:
                    data = data[from_record_info.file_record_id:]
                elif csv_file == csv_files[-1]:
                    data = data[:to_record_info.file_record_id]
                else:
                    data = data[:]
                data_collection.append(data)
            
            df = pd.concat(data_collection)
            batch = self.perform_preprocessing(df)
            batch = batch.reshape([self.batch_size, self.window_size, self.feature_dim, 1])

        batch = np.asarray(batch).astype('float32')
        return batch, batch
    
    def get_file_path(self, index):
        return self.file_names[0]
    
    @staticmethod
    def perform_preprocessing(df):
        df = df.fillna(0).to_numpy()[:, 1:] 
        df /= np.max(np.abs(df), axis=0)
        return df
              
    @staticmethod
    def extract_number_of_records(meta_info_file_path):
        with open(Path(meta_info_file_path), "r") as f:
            text = f.read()
        return int(text.split('\n')[1].split('=')[1])
    
    @staticmethod
    def download_file(bucket_name, file_path, local_output_path):
        s3 = boto3.resource('s3')
        file_name = file_path.split('/')[-1]
        try:
            s3.Bucket(bucket_name).download_file(
                str(file_path), str(Path(local_output_path, file_name))
            )
        except ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
```

There are several things to note here:

Based on the Keras Sequence class we have two methods that are required to be implemented: 
* \_\_getitem\_\_ method - this method is responsible for providing a batch of data points when called. Hence it loads the chunk of the data from the different CSV files and brings them together in a batch. As a return value it provides (batch, batch), so the same data for input and target, which is due to the nature of the auto encoder model we want to train based on the data provided by the DataGenerator
* \_\_len\_\_ method - this method simply calculates the number of batches that can be created from the overall data that is at the DataGenerators disposal. Here our meta_info files come in handy once more.

Let's briefly see these two in action:

In [ ]:
download_file(bucket, f"data/ek60/tabular/2015-03-01.csv", Path("data/ek60"))
download_file(bucket, f"data/ek60/tabular/2015-03-02.csv", Path("data/ek60"))
download_file(bucket, f"data/ek60/tabular/2015-03-03.csv", Path("data/ek60"))

In [ ]:
from autoencoder import DataGeneratorFromCSV
file_names = [Path("data/ek60", el) for el in ['2015-03-01.csv', '2015-03-02.csv', '2015-03-03.csv']]
file_names

In [ ]:
data_generator = DataGeneratorFromCSV(file_names, window_size=8, feature_dim=1600, batch_size=32)
data_generator

In [ ]:
data_generator.__len__()

In [ ]:
data_generator.__getitem__(1)[0].shape

Besides the required methods we also added other methods for our convenience and for keeping track of which data points from the files to use for a batch:
* setup_record_book method - the output provided by this method is a table that contains then necessary information to keep track of the data points in play and ensure the right ones are selected from the different files for each batch. It contains a gloabl index for the data points, the index of the data point within each file as well as the file name in which the data point is stored.
* perform_preprocessing method - as the name says this method performs preprocessing operations on our data. It currently simply fills NA values with 0 and create a numpy array from the dataframe but in general it can be extended to perform further cleanup and data selection to improve model quality.

<a id='Frameworks'></a>
## Leveraging SageMaker for Deep Learning Frameworks

The parts described so far can be used on your local machine to train a model using TensorFlow. But we aim to train our model on AWS SageMaker to leverage the compute and storage capabilities that make this service so powerful. In order to do this we need to wrap this code into a script that fulfills some basic formalities. In this way you can make deep learning frameworks like TensorFlow, PyTorch or MXNet run on AWS SageMaker. This is done using the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/index.html). The code snippets necessary to provide the connection between your "local" implementation and the format SageMaker needs can be found [here](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script). For us it looks as follows:

```python
import argparse
import json
import os
from pathlib import Path

def _parse_args():
    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--model_dir', type=str)
    parser.add_argument('--sm-model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAINING'))
    parser.add_argument('--hosts', type=list, default=json.loads(os.environ.get('SM_HOSTS')))
    parser.add_argument('--current-host', type=str, default=os.environ.get('SM_CURRENT_HOST'))
    parser.add_argument(
        '--epochs',
        type=int,
        default=10,
        help='The number of steps to use for training.')
    parser.add_argument(
        '--batch-size',
        type=int,
        default=128,
        help='Batch size for training.')
    parser.add_argument(
        '--window-size',
        type=int,
        default=8,
        help='Window size - number of records to process as one.')

    return parser.parse_known_args()


if __name__ == "__main__":
    args, unknown = _parse_args()
    local_train_data_path = args.train
    train_data_file_list = sorted(
        list(Path(local_train_data_path).glob('**/*.csv')), 
        key=lambda file_path: file_path.stem
    )
    data_generator = DataGeneratorFromCSV(
        train_data_file_list, 
        window_size=args.window_size, 
        feature_dim=1600, 
        batch_size=args.batch_size
    )
    
    autoencoder = model(
        data_generator, 
        data_generator, 
        args.epochs, 
        window_size=args.window_size, 
        feature_dim=1600, 
        batch_size=args.batch_size
    )

    if args.current_host == args.hosts[0]:
        # save model to an S3 directory with version number '00000001' in Tensorflow SavedModel Format
        # To export the model as h5 format use model.save('my_model.h5')
        autoencoder.save(os.path.join(args.sm_model_dir, '000000001'))

```

In the *\_parse_args()* method we have the environment variables that are provided by SageMaker, including the hyperparameters we can provide to our script. Their individual meaning is explained in the [docs](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script) in more detail. The parsing happens in its usual Python boilerplate fashion. 

In the *\_\_main\_\_* section we then only implement the few necessary steps to start the model training. SageMaker already downloaded the training data from S3 to the training instance's local hard drive and from there we get the file paths of the CSV files. We use them to instantiate our DataGenerator and then provide the generator as input as well as output to the model. Finally the trained autoencoder model is saved to disk from where SageMaker will copy it over to S3.

> Exercise: 

> In this implementation some parameters are still hard coded: 
> * bucket name
> * feature_dim of the data

> Make them parameters that you can provide via the SageMaker Estimator call instead of within the *autoencoder.py* file.

Now we have all the pieces in place and go on to the few lines of code still remaining to start the training job to actually create our model.

<a id='TrainingJob'></a>
## Creating the model

For model training we follow the same steps as with the Random Cut Forest model. The main difference comes from the fact that we are using not a built-in algorithm but instead rely on Tensorflow for creating the estimator object. When using Tensorflow on AWS SageMaker you need to provide an "entry_point" file that includes your code with the steps necessary to train the model. The nice part about this is, that the code you develop for this entry point file is very much the same as you would do when using Tensorflow on your local machine. The code snippets we discussed so far - model architecture, data generator and the necessary formalities in the \_\_main\_\_ scope - form the "entry_point" file we will provide to the AWS SageMaker Tensorflow estimator. So in the following section we will focus on the remaining surrounding parts to start a SageMaker training job using TensorFlow.

### Training a Tensorflow Model on AWS SageMaker

Similar to the RCF approach we can start the SageMaker machinery and the set the parameters regarding our training data and hyperparameters.

To test and develop this we recommend using the so called **local mode**, which will start docker containers on your SageMaker Notebook/Studio instance to simulate the training instance without all the backend overhead and thus allows much faster development feedback cycles. How to use it is described in the [docs](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode) and this [example](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/chainer_mnist/chainer_mnist_local_mode.ipynb).

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

In [ ]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [ ]:
bucket = 'gdsc4-eu'
prefix = 'data/ek60/tabular/2015'
training_data_uri = f's3://{bucket}/{prefix}'

batch_size = 128
window_size = 64
epochs = 4

hyperparameters = {'epochs': epochs, 'batch-size': batch_size, 'window-size': window_size}

Next we create the estimator object using the SageMaker Tensorflow implemenation. Notice again the need to set "role" but also the possibility to set "instance_count", "instance_type", "hyperparameters" and the essential "entry_point". 

In [ ]:
autoencoder_estimator = TensorFlow(
    entry_point='autoencoder.py',
    role=role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    framework_version='2.3.1',
    image_uri='763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04',
)

<span style="color:red">CAUTION: DO NOT EXECUTE THIS NEXT CELL UNLESS YOU HAVE PLENTY OF TIME (~1 HOUR) ON YOUR HANDS</span>

In [ ]:
#model= autoencoder_estimator.fit(training_data_uri)

The training process should be monitored in order to see whether your model is actually training, so improving from a theoretical point of view, e.g. via the value of the loss function decreasing. This can be done via the logs, collected via [AWS CloudWatch](https://aws.amazon.com/cloudwatch/?nc1=h_ls). For the training jobs you ran the logs can be reached via the SageMaker Console.

> Exercise: during training we can collect important information about the learning progress of our model. Tensorboard is a very good way to collect and visualize those metrics. Setup TensorBoard to have an easy way to monitor your training jobs progress

> Hint: an example can be found in this [notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorboard_keras/tensorboard_keras.ipynb) and this [blog article](https://aws.amazon.com/blogs/machine-learning/visualizing-tensorflow-training-jobs-with-tensorboard/)    

To improve your training result you can experiment with techniques like [early stopping](https://keras.io/api/callbacks/early_stopping/). As the name suggests "early stopping" tells trainig job to stop once the model is not getting better with additional steps.

> Exercise: implement a call back for early stopping to get the best out of your training jobs

<a id='Inference'></a>
## Inference

With our model trained and stored in S3 we now want to deploy the model to create an endpoint for us to use for predictions. In the case of our autoencoder model this means we will use the endpoint to have it reconstruct our input data.

To create the endpoint we will need the necessary execution role to grant the needed permissions for our operation.

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In case you trained your model some time before and just want to use it now, e.g. to create and endpoint out of it, you need to add the following step into your workflow. You create the [TensorflowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) object from the information stored on S3 during training. The S3 path for the *model_data* parameter can be seen in at the bottom of the training job overview in the Sagemaker Console -> Training -> Trainig Jobs -> \<Training Job Name\>.
> Exercise: find a way to get this information programmatically via the [AWS Python SDK (boto3)](https://aws.amazon.com/sdk-for-python/?nc1=h_ls)

The *image_uri* depends on the used framework and version. An overview of the available deep learning containers can be found [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#general-framework-containers). Note that the region needs to be adjusted and also keep an eye out for the difference on training and inference containers.

**CAUTION: ONLY EXECUTE THE FOLLOWING CELL IF YOU WANT TO USE A PREVIOUSLY TRAINED MODEL TO CREATE THE ENDPOIND**

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(
    model_data='s3://sagemaker-eu-west-1-435914187013/tensorflow-training-2021-03-01-01-55-35-932/output/model.tar.gz', 
    image_uri='763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-inference:2.3.1-cpu-py37-ubuntu18.04',
    role=role,
)

New we simply call the deploy method on our model to create the endpoint.

In [ ]:
predictor = model.deploy(
    initial_instance_count=1, 
    instance_type='ml.c5.xlarge',
)

### Using an already existing Endpoint
In case you or your team mates have already deployed an endpoint before, you can reuse it in the following way by creating a [TensorFlowPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-predictor) object.

We start by listing the existing endpoints using the [AWS Python SDK (boto3)](https://aws.amazon.com/sdk-for-python/?nc1=h_ls). You can also check the available endpoints using the browser by navigating to SageMaker Console -> Inference -> Endpoints. Once we got the name of the endpoint we can use it to create the predictor object.

In [ ]:
import boto3
from sagemaker.tensorflow import TensorFlowPredictor

client = boto3.client('sagemaker') 

endpoints = client.list_endpoints(
    StatusEquals='InService'
)
endpoints

In [ ]:
endpoint_name = endpoints['Endpoints'][0]['EndpointName']
print(f"Name of running endpoint: {endpoint_name}")

In [ ]:
predictor = TensorFlowPredictor(endpoint_name)

Now that we have our predictor, let's get the data we want to use for the predictions, which in this case means we will let the predictor recreate the input, i.e. create a reconstruction. In order to do so we need to bring the data into the same shape as we fed it to the model during training. So we load the data, perform the preprocessing and bring it into the right shape from an input data perspective but also in terms of what the endpoint expects as a format.

We will use again the convenience methods we already know to interact with the S3 bucket programmatically:

In [ ]:
import boto3
from botocore.exceptions import ClientError
from pathlib import Path

def get_file_list_from_s3(bucket_name, prefix):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    file_list = list(bucket.objects.filter(Prefix=prefix))
    return [element.key for element in file_list]

def download_file(bucket_name, file_path, local_output_path):

    s3 = boto3.resource('s3')
    file_name = file_path.split('/')[-1]
    try:
        s3.Bucket(bucket_name).download_file(
            str(file_path), str(Path(local_output_path, file_name))
        )
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

We will just download the data we want to use for the prediction. This is only a small sample, for creating predictions on the full dataset we recommend to look into [batch transform jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-model-deployment.html#ex1-batch-transform). 

In [ ]:
bucket = "gdsc4-eu"
prefix_data_2015_02 = "data/ek60/tabular/2015-02"
local_ek60_data_path = Path("data/ek60")

In [ ]:
files_2015_02 = get_file_list_from_s3(bucket, prefix_data_2015_02)
for s3_file_path in files_2015_02:
    download_file(bucket, s3_file_path, local_ek60_data_path)

Next we will load the data of a particular file (we will use the same one as in last weeks session to also compare the results between the two approaches) and then perform the identical preprocessing steps from the model training. For this we use a method that similar to the one used in the DataGenerator implementation. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
csv_file = Path("data/ek60/2015-02-22.csv")
window_size = 8
feature_dim = 1600

In [ ]:
df = pd.read_csv(csv_file, index_col=0, sep=',', parse_dates=True, header=0)

The following methods are used for the data preparation within the prediction step and also to calculate the reconstruction error, the anologon to the anomaly score we saw last week for the random cut forest (RCF) model.

In [ ]:
def create_input_datapoint(index, df, window_size, feature_dim=1600):
    df = df[index : index + window_size]
    data = perform_preprocessing(df)
    return data.reshape([1, window_size, feature_dim, 1])

def perform_preprocessing(df):
    data = df.fillna(0).to_numpy() 
    data /= np.max(np.abs(data), axis=0)
    return data

def calculate_mean_squared_error(original, reconstruction):
    return (np.square(original - reconstruction)).mean(axis=None)

Now we create the reconstructions for each timestamp by iterating over the dataframe. Each timestamp is represented by its own data plus the timestamps lying ahead according to the used *window_size*. The input format for the endpoint follows the TensorFlow Serving REST API. How this looks can be checked [here](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html#making-predictions-against-a-sagemaker-endpoint).

In [ ]:
results = {
    "timestamps": list([]),
    "reconstruction_errors": list([]),
}

for i in range(0, df.shape[0] - window_size):
    if i % 50 == 0:
        print(f"Reconstructing datapoint {i + 1} out of {df.shape[0] - window_size}")
    data_point = create_input_datapoint(i, df, window_size, feature_dim)
    prediction = predictor.predict({'instances': [data_point.tolist()]})
    reconstructed_data_point = np.array(prediction['predictions'])
    mse = calculate_mean_squared_error(data_point, reconstructed_data_point)
    results["reconstruction_errors"].append(mse)
    results["timestamps"].append(df.index[i])


From here on the steps should be very familiar since they are the same ones we performed last week for the RCF model to post-process its results and manually validate.

In [ ]:
results_df = pd.DataFrame(
        {"reconstruction_errors": results["reconstruction_errors"]}, 
        index=pd.DatetimeIndex(results["timestamps"]),
)
results_df

In [ ]:
score_mean = results_df.mean()
score_std = results_df.std()
anomaly_score_threshold = score_mean + 2 * score_std

In [ ]:
anomalies = results_df[results_df["reconstruction_errors"].values > anomaly_score_threshold.values]
anomalies

In [ ]:
ek60_file = Path("data/ek60/2015-02-22.csv") 
ek60_inference_data = pd.read_csv(
    ek60_file,
    index_col=0,
    sep=',',
    parse_dates=True,
    header=0,
)
ek60_inference_range = [float(column_name.split("_")[2].split("m")[0]) for column_name in ek60_inference_data.columns]

In the notebook kernel we are using matplotlib is not installed by default. So we need to install it:

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt   

In [ ]:
fig, ax1 = plt.subplots(figsize=(25,10))
ax2 = ax1.twinx()

ax1.pcolormesh(
    pd.to_datetime(ek60_inference_data.index),
    ek60_inference_range,
    ek60_inference_data.to_numpy().T,
    cmap=plt.get_cmap('plasma'),
    vmin=-90,
    vmax=-30
)

ax1.set_xlabel("Time")
ax1.set_ylabel("Meters")

ax2.plot(
    results_df["2015-02-22 00:00": "2015-02-22 23:59"].index,
    results_df["2015-02-22 00:00": "2015-02-22 23:59"]["reconstruction_errors"],
    color="whitesmoke",
    linewidth=0.4,
)
ax2.scatter(
    anomalies["2015-02-22 00:00": "2015-02-22 23:59"].index, 
    anomalies["2015-02-22 00:00": "2015-02-22 23:59"]["reconstruction_errors"], 
    color="black"
)
ax2.set_ylabel("Anomaly Score")

fig.suptitle("EK60 Echosounder - Anomaly Candidates in Echogram")

As in the session last week we added the anomaly scores (this time called reconstruction error) over time on top of the echogram. The thin grey time series indicates the anomaly score provided by the autoencoder model. Anomaly scores with black dots indicate an anomaly score above our threshold. Those are of particular interest and we want to check if at this time we also see an interesting structure in the echogram. Also we want to validate if the anomaly score values (their tendency to be lower or higher) match our human perception of "things being out of the ordinary in the echogram".

This time it looks like bad news on a first glance for our first attempt to train an autoencoder model to detect the anomalies in our data. There is an increase in the reconstruction error/anomaly score for the time with increase activity at the ocean surface but this seems not to be enough to be seen as an outlier. The brighter sprincles for the most part also increase the reconstruction error, but also by far not enough to be classified as an outlier. However, the statistics for the threshold were only calculated on this tiny part of the data.

Nonetheless, this means back to EDA and further analyzing the model results to gain better understanding on the model outputs!

> Exercise: We only used one day of inference data to compute our statistics for setting the threshold. Scale this out and recheck the findings for this echogram - are more data points you would consider anomolous classified as such by the model now?

> Exercise: Visualize the original input data vs. the reconstruction in order to gain insights whether our model has problems in general or with particular structures in the data. Hint: check the blog article on [Convolutional Autoencoders for Image Noise Reduction](https://towardsdatascience.com/convolutional-autoencoders-for-image-noise-reduction-32fce9fc1763) for a way how to do this

> Exercise: Try improving the model performance by e.g. 
> * using a pre-trained CNN for the encoder part
> * introducing regularization in your model via [drop-out layers](https://keras.io/api/layers/regularization_layers/dropout/)

<span style="color:red">BEFORE WE FINISH UP, MAKE SURE TO STOP YOUR ENDPOINT</span>

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(predictor.endpoint)

***

<a id='Epilogue'></a>
# Wrap Up

## Recap
In today's session we looked into a possible way to combine the data sets from each sensor to produce a comprehensive data point containing all the information at hand for the given point in time.

Our main focus however was to introduce the steps necessary to train a convolutional autoencoder model using TensorFlow and using it for inference by reconstructing our input data and computing the reconstruction error to detect anomalies in our data. This covered the steps: 
* defining the model architecture with its different layers
* setting up the data generator to fed bigger amounts of data to the model
* wrapping the TensorFlow code in a format necessary for AWS SageMaker
* creating the SageMaker training job
* creating the endpoint - right from the just trained model or by using a previously trained model
* reconstructing our data using the inference endpoint
* calculating the reconstruction error and setting the threshold value
* identifying potential anomalous data points

## Epilogue
This is the end of the tutorial sessions for the Global Data Science Challenge. We hope you had fun following along with the tutorials, learned some new skills and are now eager to experiment with the data leveraging the introduced concepts. We wish you best of luck in the challenge! 